## An Introduction to Dask on Ray

Ray library comes pre-installed on this notebook image:

In [2]:
import os
import ray
import ray.util
from ray.util.client import ray as rayclient

Connect to the Ray Server,
Unless we are already connected

In [3]:
headhost = os.environ['RAY_CLUSTER']

if not rayclient.is_connected():
    ray.util.connect('{ray_head}:10001'.format(ray_head=headhost))

In [4]:
from ray.util.dask import ray_dask_get
import dask
import dask.array as da
import dask.dataframe as dd
import numpy as np
import pandas as pd

In [5]:
random_data = da.from_array(np.random.randint(0, 1000, size=(256, 256)))

You can tell dask to use the ray scheduling backend for specific computations:

In [6]:
random_data.mean().compute(scheduler=ray_dask_get)

500.7792663574219

Alternatively, you can configure dask to use ray by default:

In [7]:
dask.config.set(scheduler=ray_dask_get)

In [8]:
data = dask.datasets.timeseries()
data.head(5)

,id,name,x,y
timestamp,,,,
2000-01-01 00:00:00,997,Hannah,-0.703919,0.755042
2000-01-01 00:00:01,1023,George,-0.486810,-0.156185
2000-01-01 00:00:02,1014,Ray,-0.153355,-0.815755
2000-01-01 00:00:03,1059,Norbert,-0.757544,-0.976137
2000-01-01 00:00:04,997,Edith,-0.855295,-0.714161


In [9]:
from sklearn.linear_model import LinearRegression
def train(partition):
    est = LinearRegression()
    est.fit(partition[['x']].values, partition.y.values)
    return est

In [10]:
models_by_name = data.groupby('name') \
    .apply(train, meta=object) \
    .compute() \
    .sort_index()
models_by_name

name
Alice       LinearRegression()
Bob         LinearRegression()
Charlie     LinearRegression()
Dan         LinearRegression()
Edith       LinearRegression()
Frank       LinearRegression()
George      LinearRegression()
Hannah      LinearRegression()
Ingrid      LinearRegression()
Jerry       LinearRegression()
Kevin       LinearRegression()
Laura       LinearRegression()
Michael     LinearRegression()
Norbert     LinearRegression()
Oliver      LinearRegression()
Patricia    LinearRegression()
Quinn       LinearRegression()
Ray         LinearRegression()
Sarah       LinearRegression()
Tim         LinearRegression()
Ursula      LinearRegression()
Victor      LinearRegression()
Wendy       LinearRegression()
Xavier      LinearRegression()
Yvonne      LinearRegression()
Zelda       LinearRegression()
dtype: object

In [11]:
alice = models_by_name.at['Alice']
alice.predict([[0]])

array([0.00637676])